In [7]:
import json
import requests
from io import BytesIO
from bs4 import BeautifulSoup as bs
from PIL import Image
import datetime
from urllib import parse
import re
import random
import time
"""
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import  expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.common.exceptions import TimeoutException
"""

jintian=datetime.datetime.now().strftime('%Y-%m-%d')
rubtime=jintian+"11:13"
rubkey=[]

s = requests.Session()

class JD:                             #定义了jd的函数class 类，可以集成操作
    def __init__(self, username, password):
        self.username = username
        self.password = password
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:56.0) Gecko/20100101 Firefox/56.0',
            'Referer': 'https://www.jd.com/'
        }

    def get_login_data(self):
        url = 'https://passport.jd.com/new/login.aspx'
        html = s.get(url, headers=self.headers).content
        soup = bs(html, 'lxml')
        display = soup.select('#o-authcode')[0].get('style')
        print("display:",display)
        auth_code = ''
        if not display:
            print('需要验证码。。。')
            auth_code_url = soup.select('#JD_Verification1')[0].get('src2')
            print(auth_code_url)
            auth_code = self.get_auth_img(auth_code_url)
        
        uuid = soup.select('#uuid')[0].get('value')
        urleid='https://passport.jd.com/uc/login'
        urleidres=s.get(urleid).content
        eidsoup=bs(urleidres,'lxml')
        eid = eidsoup.find(id='eid')['value']
        
        print('eid:::',eid) #也是被隐藏的，爬取不到
        fp = soup.select('input[name="fp"]')[0].get('value')  # session id
       
        # print(fp)  也是隐藏的，爬取不到，但是不影响登录
        _t = soup.select('input[name="_t"]')[0].get('value')  # token
        login_type = soup.select('input[name="loginType"]')[0].get('value')
        pub_key = soup.select('input[name="pubKey"]')[0].get('value')
        #print(pub_key)
        sa_token = soup.select('input[name="sa_token"]')[0].get('value')
        
        urlseqId="https://seq.jd.com/jseqf.html?bizId=passport_jd_com_login_pc&platform=js&version=1"
        htmlseqId=s.get(urlseqId).content
        soupseqId = bs(htmlseqId, 'lxml') 
        soupseqId=re.findall('var _jdtdmap_sessionId="'+'(.*?)''\"',str(soupseqId))[0]
        data = {
            'uuid': uuid,
            #'ReturnUrl':'https://a.jd.com/',
            'eid': eid,
            'fp': fp,
            '_t': _t,
            'loginType': login_type,
            'loginname': self.username,
            'nloginpwd': self.password,
            #'chkRememberMe': True,
            'authcode': auth_code,
            'pubKey': pub_key,
            'sa_token': sa_token,
            'seqSid':soupseqId,
            'useSlideAuthCode':1
        }
        
        for values in data.values():
            print(values)
        return data

    def get_auth_img(self, url):
        auth_code_url = 'http:' + url
        print(auth_code_url)
        auth_img = s.get(auth_code_url, headers=self.headers)
        if auth_img.status_code == 200:
            file = BytesIO(auth_img.content)
            img = Image.open(file)
            img.show() 
        code = input('请输入验证码：')
        return code

    def login(self):
        url = 'https://passport.jd.com/uc/loginService'
        data = self.get_login_data()
        #print("构造的数据：",data)
        headers = {
            'Referer': 'https://passport.jd.com/uc/login?ltype=logout',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:56.0) Gecko/20100101 Firefox/56.0',
            'X-Requested-With': 'XMLHttpRequest'
        }
        content = s.post(url, data=data, headers=headers)
        print(content.text)
        result = json.loads(content.text[1: -1])
        print("result:",result)   
        if result.get('success'):
            print('登录成功')
        else:
            print('登录失败')
        #这个只能抢一些券，先前把券的key 输入进去才行
        #url='https://a.jd.com/indexAjax/getCoupon.html?callback=jQuery5342064&key=fc2aaa4c4303a670b9623513b3b82437dc4e5b5cb7a5509bf73248ee4cc0c7f4f0687871809bc488d7a06e7787f1929b&type=1&_=1518077546111'
        rubkey=jd.rubkey()
        
        for keys in rubkey:
            jQuery='jQuery{}'.format(random.randint(4000000,4999999)) # 生成随机整数
            data={'callback':jQuery,
                  'key':keys,
                  'type':'1'
                  }
            query=parse.urlencode(data)
            url='https://a.jd.com/indexAjax/getCoupon.html?{}'.format(query)
            time.sleep(random.uniform(3,6))  #这个意思是随机3到6秒的小数
            headers1={'Host':'a.jd.com',
                    'Referer':'https://a.jd.com/',
                    'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36',
                    'X-Requested-With':'XMLHttpRequest'}
            print('抢券等待中....到时间就抢，莫急')
           
            jQuery1='jQuery{}'.format(random.randint(1000000,1999999))
            data1={'callback':jQuery1,
                   'method':'Login'}
            query1=parse.urlencode(data1)
            url1='https://passport.jd.com/loginservice.aspx?{}'.format(query1)
            resp=s.get(url1,headers=self.headers).text
            jdata1=re.findall(jQuery1+'\((.*?)\)',resp)[0]  #这个含义是把data清洗出来，成json格式
            jdata1=json.loads(jdata1)
         
            if jdata1['Identity']['Unick']==self.username:
                print('再次校验登录成功')
                while True:   #定时抢券，先进行排队时间
                    now=datetime.datetime.now().strftime('%Y-%m-%d%H:%M')
                    if now==rubtime:
                        respo=s.get(url,headers=headers1)
                        result=respo.text
                        print(result)
                        break
            else:
                print('再次校验登录失败')
                exit() 
        
    def rubkey(self):
        for pagenum in range(1,4):
            JQuery='jQuery{}'.format(random.randint(4000000,4999999))
            data={'callback':JQuery,
                    'catalogId':'0',
                    'page':pagenum,
                    'pageSize':9}
            querydata=parse.urlencode(data)
            url='https://a.jd.com/indexAjax/getCouponListByCatalogId.html?{}'.format(querydata)
            headers={'Host':'a.jd.com',
                    'Referer':'https://a.jd.com/',
                    'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36',
                    'X-Requested-With':'XMLHttpRequest'}
            res11=s.get(url,headers=headers).text
          
            jdata=re.findall(JQuery+'\((.*?)\)',res11)[0]  #这个含义是把data清洗出来，成json格式
         
            jdata=json.loads(jdata) #这个含义把jdata转化为dict形式

            #for key in jdata.keys():  #这个是遍历jdata的键值，并打印出来，挺有用
            #print(key)

            new=jdata['couponList'] #这个语句的意思是把jdata这个dict的couponList的键值提取出来
            for rub in new:
                rubkey.append(rub['ruleKey'])
        print("此次共抢{}个券".format(len(rubkey)))
        haoquan=[rubkey[8:16]]
        return rubkey  #返回rubkey的字典格式

username = 'saant'
password = 'xuei'
jd = JD(username, password)
jd.login()





display: None
需要验证码。。。
None


TypeError: can only concatenate str (not "NoneType") to str